In [1]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")  
model.export(format="onnx", opset=14) # преобразование в onnx

Ultralytics 8.3.182 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon Processor (Cascadelake))
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from 'yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (5.4 MB)

ONNX: starting export with onnx 1.17.0 opset 14...
============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

ONNX: slimming with onnxslim 0.1.64...
ONNX: export success ✅ 2.1s, saved as 'yolo11n.onnx' (10.2 MB)

Export complete (2.6s)
Results saved to /home/jupyter/work/resources/transform
Predict:         yolo predict task=detect model=yolo11n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolo11n.onnx imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


'yolo11n.onnx'

In [2]:
import onnx
from onnx_tf.backend import prepare

onnx_path = "yolo11n.onnx"   # путь к ONNX
onnx_model = onnx.load(onnx_path)
tf_rep = prepare(onnx_model, strict=False)
tf_rep.export_graph("yolo_saved_model")      # создание директории saved_model


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

INFO:absl:Function `__call__` contains input name(s) x, y with unsupported characters which will be renamed to onnx_tf_prefix__model_23_add_1_x, onnx_tf_prefix__model_23_mul_2_y in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: yolo_saved_model/assets


INFO:tensorflow:Assets written to: yolo_saved_model/assets
INFO:absl:Writing fingerprint to yolo_saved_model/fingerprint.pb


In [3]:
import tensorflow as tf

saved_model_dir = "yolo_saved_model"  
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# Разрешаем встроенные ops и Select TF Ops (Flex)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,    # обычные ops
    tf.lite.OpsSet.SELECT_TF_OPS       # разрешаем встраивать TF ops (Flex)
]
converter.allow_custom_ops = True

tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)

10667256